
# CGS Group Project: Predictive Power of Populations

**Collaborators: Josefina Waugh, Anne Bastian, Bridget Smart, Ebba Mark**

Preliminary toy model implementation of:
- Vote on binary outcome
- Voting agents endowed with personal views (pits: dynamic), information-processing function of social network, sensitivity to "global information/statistics."

### Defining Agents, Classes, Functions